In [1]:
import numpy as np
import pandas as pd
import os

### Download videos for training

Downloads videos as mp4 and saves the audio as mp3. mp4 may or may not contain audio due to the API being limited, though this should not matter (except for the small negative impact on file size). Currently videos are being downloaded in 720p since 1080p seems bugged

In [2]:
from pytube import YouTube 
  
urls = ["https://www.youtube.com/watch?v=Yt3-a9mExZg", "https://www.youtube.com/watch?v=cGt8bEcd9Ms", 
        "https://www.youtube.com/watch?v=rhzTeSrUynE"]
pathVideo = 'Data/Video'
pathAudio = 'Data/Audio'

for i, url in enumerate(urls):
    yt = YouTube(url)
    yt.streams.filter(abr="160kbps", progressive=False).first().download(filename="Audio"+str(i)+".mp3", output_path = pathAudio)
    yt.streams.filter(res="720p", progressive=False).first().download(filename="Video"+str(i)+".mp4", output_path= pathVideo)
##

### Split video into frames

Goes through each downloaded video. Creates a directory per video inside 'tempFrames' and then saves the frames here. We can set the amount of frames we want per minute. Note that file sizes of these images can balloon quickly, exceeding the filesize of the video itself.

In [3]:
import cv2
FRAMES_PER_MINUTE = 10
frame_counter = 60000 / FRAMES_PER_MINUTE
pathVideo = 'Data/Video/'
videoNames = []
for filename in os.listdir(pathVideo):
    f = os.path.join(pathVideo, filename)
    if os.path.isfile(f): ## checking if it is a file
        videoNames.append(filename)

for video in videoNames:
    path = pathVideo + video
    outputPath = 'Data/tempFrames/' + video[:-4] + '/' ## make directory per video
    if not os.path.exists(outputPath):
        os.mkdir(outputPath)
        print(f'Created new dir {outputPath}')
    vidcap = cv2.VideoCapture(pathVideo + video)
    success,image = vidcap.read()
    print(f'Succesful videocapture?: {success}')
    count = 0
    while success:
        cv2.imwrite(outputPath + "%d_seconds.jpg" % (count*(frame_counter/1000)), image)     # save frame as JPEG file   
        count += 1
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*frame_counter))   
        success,image = vidcap.read()
        

Succesful videocapture?: False
Succesful videocapture?: True
Succesful videocapture?: True
Succesful videocapture?: True


### Segmenting faces/webcam feeds

By segmenting individuals, we can create temporary folders containing the frames per individual participant. This might be neccessary depending on the emotion detection model that we apply. It seems better to avoid this step is possible, as it introduces an extra model (segmentation), and also requires the creation of more files which need to be operated on. This could be rather inefficient

In [4]:
## Some code here for extracting individuals, saving new cropped images

### Video emotion detection

Now we run an emotion detection model to acquire some score. For every frame, we should get some "emotion scores" per individual. We can then extract some values such as minimum, maximum and mean scores per person. We can eventually combine this into a final score. For this to be as accurate as possible we will measure audio later 

In [5]:
from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt
face_model = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
capture = cv2.VideoCapture('Data\Video\Video2.mp4')

length = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)
numberOfFrames = 500
currentFrame = 0
for i in range(2002):
    _, frame = capture.read()
    if currentFrame != numberOfFrames:
        currentFrame += 1
        continue
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_model.detectMultiScale(gray, 1.1, 5)

    for (x, y, w ,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

    for face in faces:
        print(f'frame {i}')
        print(face)
        emotion = DeepFace.analyze(frame, actions = ['emotion'])
        print(emotion[0]['dominant_emotion'])

    currentFrame = 0
    cv2.imshow('x',frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

### Audio analysis

#### Speech to text

In [ ]:
from datasets import load_dataset
from transformers import pipeline

dataset = load_dataset("anton-l/superb_demo", "er", split="session1")

classifier = pipeline("audio-classification", model="superb/wav2vec2-base-superb-er")
labels = classifier(dataset[0]["file"], top_k=5)



Found cached dataset superb_demo (C:/Users/ramon.cremers/.cache/huggingface/datasets/anton-l___superb_demo/er/1.9.0/77d23894ff429329a7fe80f9007cabb0deec321316f8dda1a1e9d10ffa089d08)


In [ ]:
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor

def map_to_array(example):
    speech, _ = librosa.load(example["file"], sr=16000, mono=True)
    example["speech"] = speech
    return example

# load a demo dataset and read audio files
dataset = load_dataset("anton-l/superb_demo", "er", split="session1")
print(dataset)
dataset = dataset.map(map_to_array)

model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-base-superb-er")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-base-superb-er")

# compute attention masks and normalize the waveform if needed
inputs = feature_extractor(dataset[:4]["speech"], sampling_rate=16000, padding=True, return_tensors="pt")

logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)
labels = [model.config.id2label[_id] for _id in predicted_ids.tolist()]


Found cached dataset superb_demo (C:/Users/ramon.cremers/.cache/huggingface/datasets/anton-l___superb_demo/er/1.9.0/77d23894ff429329a7fe80f9007cabb0deec321316f8dda1a1e9d10ffa089d08)
Loading cached processed dataset at C:\Users\ramon.cremers\.cache\huggingface\datasets\anton-l___superb_demo\er\1.9.0\77d23894ff429329a7fe80f9007cabb0deec321316f8dda1a1e9d10ffa089d08\cache-4a0c27eacd16263f.arrow


Dataset({
    features: ['file', 'audio', 'label'],
    num_rows: 6
})


In [ ]:
labels

['hap', 'hap', 'ang', 'hap']